# Accessing the emodels and running custom protocols

This notebook shows how to access the optimised emodels and how to run custom protocols on them.

## Instantiating an access point to Nexus

Please fill in the values to match the metadata of the model you want to access

In [4]:
emodel = "L5_TPC"
etype = "cAC"
mtype = None
ttype = None
iteration_tag = "XXXX-XX-XX"
species = "mouse"
brain_region = "SSCX"

nexus_project = "" # specify the nexus project to use
nexus_organisation = "" # specify the nexus organisation to use
nexus_endpoint = "" # specify the nexus endpoint to use

# Advanced settings (only change if you know what you are doing)
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project=nexus_project,
    organisation=nexus_organisation,
    endpoint=nexus_endpoint,
    forge_path=forge_path,
    etype=etype,
    mtype=mtype,
    ttype=ttype,
    iteration_tag=iteration_tag,
)

## Retrieving the emodels

In [6]:
emodels = access_point.get_emodels()

In [ ]:
import pprint

print(f"Found {len(emodels)} emodels")
print("\nExample of an emodel:\n")
pprint.pprint(emodels[0].as_dict())

## Running protocols and the emodel

Instantiate the evaluator which contains the model and the protocols used for optimisation:

In [ ]:
from bluepyemodel.evaluation.evaluation import get_evaluator_from_access_point

evaluator = get_evaluator_from_access_point(
    access_point,
    stochasticity=False,
    include_validation_protocols=False,
)

To rerun the protocols that were used for optimisation on the first of the models, do:

In [9]:
responses = evaluator.run_protocols(protocols=evaluator.fitness_protocols.values(), param_values=emodels[0].parameters)

If you wish to run additional protocols, you can first define a new protocol:

In [10]:
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration
from bluepyemodel.evaluation.evaluator import define_protocol

protocol_name = "Step_test"

stimuli = [{
    "holding_current": -0.08,
    "threshold": 0.16,
    "amp": 300,
    "delay": 100,
    "duration": 1000,
    "totduration": 1200
}]

recordings = [
    {
        "type": "CompRecording",
        "name": f"{protocol_name}.soma.v",
        "location": "soma",
        "variable": "v",
    }
]

my_protocol_configuration = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

my_protocol = define_protocol(my_protocol_configuration)

Note that the name of the protocol matters as BluePyEmodel will use it to detect which shape the protocol should have. For example, here, "step" is in the name and therefore a step protocol will be generated. A complete list of the eCode that can be used to create protocols can be seen [here](https://github.com/BlueBrain/BluePyEModel/blob/main/bluepyemodel/ecode/__init__.py). For more complex eCode, contact the package authors for an explanation of how to use them.

Once the protocol is created, it can be added to the protocols already in the evaluator:

In [11]:
evaluator.fitness_protocols["main_protocol"].protocols[protocol_name] = my_protocol
evaluator.fitness_protocols["main_protocol"].execution_order = (
    evaluator.fitness_protocols["main_protocol"].compute_execution_order()
)

Finally, the responses can be plotted and saved in a pdf in the current working directory:

In [ ]:
from bluepyemodel.emodel_pipeline import plotting
from bluepyemodel.emodel_pipeline.plotting import get_recording_names

responses = evaluator.run_protocols(protocols=evaluator.fitness_protocols.values(), param_values=emodels[0].parameters)
stimuli = evaluator.fitness_protocols["main_protocol"].protocols
recording_names = get_recording_names(
    access_point.get_fitness_calculator_configuration().protocols,
    stimuli,
)
plotting.traces(model=emodels[0], responses=responses, recording_names=recording_names, stimuli=stimuli, figures_dir="./")